In [64]:
import glob
import pandas as pd

In [42]:
human_story_details = glob.glob("./data/v3/*/*.json")

In [77]:
def fetch_story(story_id):
    if not isinstance(story_id, str):
        return None
    human_stories = glob.glob("./data/v3/*/*.md")
    path = [path for path in human_stories if story_id in path][0]
    f = open(path, 'r')
    return f.read()

In [123]:
dfs = []
for path in human_story_details:
    df = pd.read_json(path).T
    dfs.append(df)
    
df = pd.concat(dfs).reset_index().rename(columns={'index': 'key'})
df = df[df["key"] != "gpt4"]
df["story_id"] = df["url"].str.extract('/comment/(\w+)')
df["text"] = df["story_id"].apply(fetch_story)

In [80]:
df.to_csv("./data/human_stories.csv")

In [126]:
human_df = pd.read_csv("./data/human_stories.csv", encoding='cp1252')

In [85]:
llm_stories = glob.glob("./llm_story_generation_results_v1/*/*/*")

In [162]:
model_map = {
    "meta-llama_Llama-2-7b-chat-hf": "Llama-2-7B",  
    "meta-llama_Llama-2-13b-chat-hf": "Llama-2-13B", 
    "lmsys_vicuna-33b-v1.3": "Vicuna-33B", 
    "meta-llama_Llama-2-70b-chat-hf": "Llama-2-70B",
    "gpt-4": "GPT-4"    
}

In [179]:
dfs = []
for path in llm_stories:
    df = pd.read_json(path, typ='series').to_frame().T
    df["strategy"] = path.split("\\")[-2]
    df["story_id"] = path.split("\\")[-1].split("_")[-1].replace(".json", "")
    dfs.append(df)
        
llm_df = pd.concat(dfs)
llm_df["model_short"] = llm_df["model_name"].apply(model_map.get)
llm_df["characters"] = llm_df["characters"].str.strip()
llm_df["output"] = llm_df["output"].str.strip()
llm_df = llm_df.rename(columns={
    "output": "text",
    "story_prompt": "prompt"
})
llm_df = llm_df.drop(columns="id")

In [180]:
llm_df.to_csv("./data/llm_stories.csv")

In [181]:
llm_df.columns

Index(['model_name', 'prompt', 'characters', 'text', 'strategy', 'story_id',
       'model_short'],
      dtype='object')

In [182]:
human_df.columns

Index(['story_id', 'prompt', 'text', 'net_upvotes', 'model_short'], dtype='object')

In [183]:
df = pd.concat([human_df, llm_df])

In [184]:
df.to_csv("./data/stories.csv")